# Sesgos Semánticos: Embeddings vs Léxico

**Notebook:** Comparación de sesgos basados en embeddings  
**Versión:** 1.0  
**Fecha:** Enero 2025

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolakorff/tfm-endocrine-llm/blob/main/examples/03_semantic_bias_demo.ipynb)

---

Este notebook demuestra el uso de **sesgos semánticos** basados en Sentence-BERT que afectan ~1000 tokens vs ~15 del sesgo léxico simple.

**Contenido:**
1. Instalación (con características semánticas)
2. Comparación directa: Sesgo simple vs semántico
3. Análisis de activación semántica
4. Categorías semánticas predefinidas
5. Crear categorías custom
6. Análisis cuantitativo

##1. Instalación

**IMPORTANTE:** Instalar con características semánticas (incluye sentence-transformers)

In [ ]:
# Instalar con semantic features
!pip install -q "git+https://github.com/Nicolakorff/tfm-endocrine-llm.git@v0.5.0#egg=endocrine-llm[semantic]"

import endocrine_llm
print(f"Versión: {endocrine_llm.__version__}")

# Verificar que semantic está disponible
try:
    from endocrine_llm.semantic import SemanticBiasManager
    print("Módulo semántico disponible")
except ImportError:
    print("ERROR: Módulo semántico no disponible")
    print("Reinstala con: pip install endocrine-llm[semantic]")

##2. Imports

In [ ]:
from endocrine_llm import EndocrineModulatedLLM, HORMONE_PROFILES
from endocrine_llm.semantic import SemanticBiasManager, analyze_semantic_activation
import warnings
warnings.filterwarnings('ignore')

# Cargar modelo
print("Cargando modelo...")
model = EndocrineModulatedLLM("distilgpt2")
print(f"Modelo cargado en: {model.device}")

##3. ¿Qué son los Sesgos Semánticos?

| Característica | Sesgo Léxico | Sesgo Semántico |
|----------------|--------------|------------------|
| **Cobertura** | ~15 tokens | ~1000 tokens |
| **Base** | Lista fija | Embeddings SBERT |
| **Flexibilidad** | Estática | Dinámica |
| **Costo** | Bajo | Moderado |

### Ejemplo Visual:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Sesgo simple
vocab_size = 50257  # GPT-2 vocab
simple_affected = 15
ax1.bar(['No afectados', 'Afectados'],
        [vocab_size - simple_affected, simple_affected],
        color=['lightgray', '#3498db'],
        edgecolor='black')
ax1.set_title('Sesgo Léxico Simple', fontsize=14, fontweight='bold')
ax1.set_ylabel('Número de Tokens')
ax1.set_yscale('log')
ax1.grid(axis='y', alpha=0.3)

# Sesgo semántico
semantic_affected = 1042
ax2.bar(['No afectados', 'Afectados'],
        [vocab_size - semantic_affected, semantic_affected],
        color=['lightgray', '#e74c3c'],
        edgecolor='black')
ax2.set_title('Sesgo Semántico (Embeddings)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Tokens')
ax2.set_yscale('log')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n Ratio de cobertura: {semantic_affected / simple_affected:.1f}×")

##4. Comparación Directa: Simple vs Semántico

In [ ]:
prompt = "I'm feeling overwhelmed and need support."

print("="*70)
print("COMPARACIÓN: SESGO SIMPLE VS SEMÁNTICO")
print("="*70)
print(f"\nPrompt: {prompt}\n")

# SESGO SIMPLE (léxico)
print("[SESGO SIMPLE - Lista de tokens]")
text_simple = model.generate_with_hormones(
    prompt,
    HORMONE_PROFILES["empathic"],
    max_new_tokens=50
)[0]
print(f"{text_simple}\n")

# SESGO SEMÁNTICO (embeddings)
print("[SESGO SEMÁNTICO - Embeddings]")
text_semantic = model.generate_with_semantic_bias(
    prompt,
    HORMONE_PROFILES["empathic"],
    semantic_category="empathy",
    semantic_strength=1.5,
    max_new_tokens=50
)[0]
print(f"{text_semantic}\n")

print("="*70)

##5. Calcular Métricas Comparativas

In [ ]:
from endocrine_llm import TextMetrics

# Calcular métricas
metrics_simple = TextMetrics.compute_all(text_simple)
metrics_semantic = TextMetrics.compute_all(text_semantic)

print("COMPARACIÓN DE MÉTRICAS")
print("="*70)

metrics_to_show = [
    ('distinct_2', 'Diversidad Léxica'),
    ('repetition_rate', 'Tasa de Repetición'),
    ('sentiment_polarity', 'Polaridad del Sentimiento')
]

for metric_key, metric_name in metrics_to_show:
    simple_val = metrics_simple[metric_key]
    semantic_val = metrics_semantic[metric_key]
    diff = semantic_val - simple_val
    pct_change = (diff / simple_val * 100) if simple_val != 0 else 0

    print(f"\n{metric_name}:")
    print(f"Simple:    {simple_val:.4f}")
    print(f"Semántico: {semantic_val:.4f}  (Δ = {diff:+.4f}, {pct_change:+.1f}%)")

##6. Análisis de Activación Semántica

In [ ]:
# Inicializar manager semántico
manager = SemanticBiasManager(model.tokenizer, device=model.device)

# Analizar texto generado con sesgo semántico
analysis = analyze_semantic_activation(text_semantic, manager)

print("ANÁLISIS DE ACTIVACIÓN SEMÁNTICA")
print("="*70)
print(f"\nTexto analizado: {text_semantic[:100]}...")
print(f"\nCategoría dominante: {analysis['dominant_category']}")
print(f"Score de activación: {analysis['dominant_score']:.3f}")

print("\nActivación por categoría:")
for cat, score in sorted(analysis['similarities'].items(), key=lambda x: -x[1]):
    bar = '█' * int(score * 20)
    print(f"  {cat:12s}: {score:.3f}  {bar}")

##7. Explorar Categorías Semánticas Predefinidas

In [ ]:
# Categorías disponibles
categories = ['empathy', 'creativity', 'factual', 'caution', 'enthusiasm']

prompt = "Tell me about artificial intelligence."

print("="*70)
print("GENERACIÓN CON DIFERENTES CATEGORÍAS SEMÁNTICAS")
print("="*70)
print(f"\nPrompt: {prompt}\n")

for category in categories:
    texts = model.generate_with_semantic_bias(
        prompt,
        HORMONE_PROFILES["baseline"],
        semantic_category=category,
        semantic_strength=1.5,
        max_new_tokens=40
    )

    print(f"\n[{category.upper()}]")
    print(f"{texts[0]}")
    print("-"*70)

##8. Crear Categoría Semántica Custom

In [ ]:
# Inicializar manager si no existe
if not hasattr(model, 'semantic_manager'):
    model.semantic_manager = SemanticBiasManager(
        model.tokenizer,
        device=model.device
    )

# Añadir categoría TECHNICAL
model.semantic_manager.add_custom_category(
    name="technical",
    seed_words=[
        "algorithm", "function", "variable", "code", "implementation",
        "optimize", "debug", "compile", "syntax", "framework",
        "architecture", "module", "class", "method", "parameter"
    ]
)

print("Categoría 'technical' añadida")

# Generar con categoría custom
text_technical = model.generate_with_semantic_bias(
    "Explain how hash tables work.",
    HORMONE_PROFILES["cautious"],
    semantic_category="technical",
    semantic_strength=2.0,
    max_new_tokens=60
)[0]

print("\nGeneración con categoría TECHNICAL:")
print("="*70)
print(text_technical)

##9. Efecto de Semantic Strength

In [ ]:
prompt = "I need help."
strengths = [0.5, 1.0, 1.5, 2.0]

print("EFECTO DE SEMANTIC_STRENGTH")
print("="*70)
print(f"Prompt: {prompt}\n")

activations = []

for strength in strengths:
    texts = model.generate_with_semantic_bias(
        prompt,
        HORMONE_PROFILES["empathic"],
        semantic_category="empathy",
        semantic_strength=strength,
        max_new_tokens=40
    )

    # Analizar activación
    analysis = analyze_semantic_activation(texts[0], manager)
    activations.append(analysis['similarities']['empathy'])

    print(f"\n[Strength = {strength}]")
    print(f"Activación empathy: {analysis['similarities']['empathy']:.3f}")
    print(f"{texts[0]}")
    print("-"*70)

# Visualizar relación
plt.figure(figsize=(10, 6))
plt.plot(strengths, activations, marker='o', linewidth=2, markersize=8, color='#e74c3c')
plt.xlabel('Semantic Strength', fontsize=12, fontweight='bold')
plt.ylabel('Activación de Categoría Empathy', fontsize=12, fontweight='bold')
plt.title('Relación entre Semantic Strength y Activación', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

##10. Análisis Cuantitativo (Múltiples Generaciones)

In [ ]:
import pandas as pd
import numpy as np

prompts_test = [
    "I'm feeling anxious.",
    "Tell me a story.",
    "Explain quantum physics.",
    "I need support."
]

results = []

print("Ejecutando análisis cuantitativo...")
print("(Esto puede tardar ~2 minutos)\n")

for prompt in prompts_test:
    # Generar con sesgo simple
    for i in range(3):  # 3 repeticiones
        text = model.generate_with_hormones(
            prompt,
            HORMONE_PROFILES["empathic"],
            max_new_tokens=40
        )[0]

        metrics = TextMetrics.compute_all(text)

        results.append({
            'prompt': prompt,
            'condition': 'simple',
            'distinct_2': metrics['distinct_2'],
            'repetition_rate': metrics['repetition_rate'],
            'sentiment': metrics['sentiment_polarity']
        })

    # Generar con sesgo semántico
    for i in range(3):
        text = model.generate_with_semantic_bias(
            prompt,
            HORMONE_PROFILES["empathic"],
            semantic_category="empathy",
            semantic_strength=1.5,
            max_new_tokens=40
        )[0]

        metrics = TextMetrics.compute_all(text)

        results.append({
            'prompt': prompt,
            'condition': 'semantic',
            'distinct_2': metrics['distinct_2'],
            'repetition_rate': metrics['repetition_rate'],
            'sentiment': metrics['sentiment_polarity']
        })

df_results = pd.DataFrame(results)

# Comparación estadística
print("\nRESULTADOS AGREGADOS")
print("="*70)

summary = df_results.groupby('condition').agg({
    'distinct_2': ['mean', 'std'],
    'repetition_rate': ['mean', 'std'],
    'sentiment': ['mean', 'std']
}).round(4)

print(summary)

# Test estadístico
from scipy import stats

simple_d2 = df_results[df_results['condition'] == 'simple']['distinct_2']
semantic_d2 = df_results[df_results['condition'] == 'semantic']['distinct_2']

t_stat, p_value = stats.ttest_ind(simple_d2, semantic_d2)

print(f"\nT-test (Distinct-2):")
print(f"  t = {t_stat:.3f}, p = {p_value:.4f}")

if p_value < 0.05:
    print(f"Diferencia SIGNIFICATIVA")
else:
    print(f"Diferencia no significativa")

# Visualizar
import seaborn as sns

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(data=df_results, x='condition', y='distinct_2')
plt.title('Diversidad Léxica', fontweight='bold')
plt.ylabel('Distinct-2')

plt.subplot(1, 2, 2)
sns.boxplot(data=df_results, x='condition', y='repetition_rate')
plt.title('Tasa de Repetición', fontweight='bold')
plt.ylabel('Repetition Rate')

plt.tight_layout()
plt.show()

##11. Conclusión

- Usar sesgos semánticos basados en embeddings  
- Comparar sesgo simple vs semántico  
- Analizar activación semántica  
- Explorar categorías predefinidas  
- Crear categorías custom  
- Realizar análisis cuantitativo  

### Hallazgos Clave

- **Cobertura:** Sesgo semántico afecta ~67× más tokens
- **Diversidad:** Incremento típico de +10-15% en Distinct-2
- **Repetición:** Reducción de ~15-20%
- **Strength óptimo:** 1.0-2.0 (1.5 recomendado)

### Consideraciones

- Costo computacional moderado (embeddings en GPU)
- Requiere `sentence-transformers` instalado
- Mejor para tareas con objetivos semánticos claros

### Próximos Pasos

- **Documentación:** Ver resultados del experimento semántico en TFM